In [ ]:
!python --version

Python 3.7.13


In [ ]:
!pip install import-ipynb
!pip install -U -q PyDrive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install sklearn-crfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# solve issue: https://github.com/TeamHG-Memex/sklearn-crfsuite/issues/60
!pip install -U 'scikit-learn<0.24'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install eli5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 7.3 MB/s 
     |████████████████████████████████| 133 kB 36.1 MB/s 
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107748 sha256=ffa8eccaa01ab7fc4d815951f24b504868a424a7d2c7ffecda0aab7971a22d94
  Stored in directory: /root/.cache/pip/wheels/cc/3c/96/3ead31a8e6c20fc0f1a707fde2e05d49a80b1b4b30096573be
Successfully built eli5
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires Jinja2<3.0,>=2.10.1, but you have jinja2 3.1.2 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is inco

In [ ]:
from google.colab import drive; drive.mount('/content/drive')

import pdb
import import_ipynb
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
MAIN_PATH = "./NLP/NER"
MODEL_PATH = f"{MAIN_PATH}/models"
DATA_PATH_MUNDI =  f"{MAIN_PATH}/data/raw/classic_mundi.csv"

### **Data**

In [ ]:
import pandas as pd
from itertools import chain
from collections import OrderedDict

In [ ]:
data = pd.read_csv(DATA_PATH_MUNDI)

In [ ]:
data.head()

,text,labels
0,Thousands of demonstrators have marched throug...,O O O O O O B-geo O O O O O B-geo O O O O O B-...
1,Iranian officials say they expect to get acces...,B-gpe O O O O O O O O O O O O O O B-tim O O O ...
2,Helicopter gunships Saturday pounded militant ...,O O B-tim O O O O O B-geo O O O O O B-org O O ...
3,They left after a tense hour-long standoff wit...,O O O O O O O O O O O
4,U.N. relief coordinator Jan Egeland said Sunda...,B-geo O O B-per I-per O B-tim O B-geo O B-gpe ...


In [ ]:
texts = [sent.split() for sent in data["text"].tolist()]
print(texts[0])

['Thousands', 'of', 'demonstrators', 'have', 'marched', 'through', 'London', 'to', 'protest', 'the', 'war', 'in', 'Iraq', 'and', 'demand', 'the', 'withdrawal', 'of', 'British', 'troops', 'from', 'that', 'country', '.']


In [ ]:
labels = [label.split() for label in data["labels"].tolist()]
print(labels[0])

['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O']


In [ ]:
from sklearn.model_selection import train_test_split

texts_sample = texts
labels_sample = labels

train_texts, val_texts, train_labels, val_labels = train_test_split(texts_sample, labels_sample , test_size=0.2, random_state=0)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.01, random_state=0)

print("train_texts:", len(train_texts))
print("test_texts:", len(val_texts))

train_texts: 37983
test_texts: 384


### **Preprocessing**

In [ ]:
def get_token_tag_sents(texts, labels):
  token_tags_sents = []
  for text, tags in zip(texts, labels):
    sent = [(token, tag) for token, tag in zip(text, tags)]
    token_tags_sents.append(sent)
  return token_tags_sents

In [ ]:
def word2features(sent, i):
    # main source: https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html

    word = sent[i][0]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
    }
    if i > 0:
        word1 = sent[i-1][0]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
        })
    else:
        features['EOS'] = True

    return features

In [ ]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

In [ ]:
train_sents = get_token_tag_sents(train_texts, train_labels)
train_sents[0]

[('Iraq', 'B-geo'),
 ("'s", 'O'),
 ('main', 'O'),
 ('Sunni', 'B-per'),
 ('Arab', 'I-per'),
 ('coalition', 'O'),
 ('has', 'O'),
 ('challenged', 'O'),
 ('partial', 'O'),
 ('vote', 'O'),
 ('results', 'O'),
 ('showing', 'O'),
 ('a', 'O'),
 ('strong', 'O'),
 ('lead', 'O'),
 ('for', 'O'),
 ('a', 'O'),
 ("Shi'ite", 'O'),
 ('coalition', 'O'),
 ('in', 'O'),
 ('last', 'O'),
 ('week', 'O'),
 ("'s", 'O'),
 ('parliamentary', 'O'),
 ('elections', 'O'),
 ('.', 'O')]

In [ ]:
val_sents = get_token_tag_sents(val_texts, val_labels)
val_sents[0]

[('But', 'O'),
 ('he', 'O'),
 ('said', 'O'),
 ('if', 'O'),
 ('that', 'O'),
 ('does', 'O'),
 ('not', 'O'),
 ('happen', 'O'),
 (',', 'O'),
 ('France', 'B-geo'),
 ('may', 'O'),
 ('propose', 'O'),
 ('sanctions', 'O'),
 ('on', 'O'),
 ('Iran', 'B-geo'),
 ("'s", 'O'),
 ('banks', 'O'),
 ('and', 'O'),
 ('insurance', 'O'),
 ('companies', 'O'),
 (',', 'O'),
 ('and', 'O'),
 ('on', 'O'),
 ('travel', 'O'),
 ('permits', 'O'),
 ('for', 'O'),
 ('specific', 'O'),
 ('people', 'O'),
 ('.', 'O')]

In [ ]:
sent2features(train_sents[0])[0]

{'+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:word.lower()': "'s",
 'BOS': True,
 'bias': 1.0,
 'word.isdigit()': False,
 'word.istitle()': True,
 'word.isupper()': False,
 'word.lower()': 'iraq',
 'word[-2:]': 'aq',
 'word[-3:]': 'raq'}

In [ ]:
print(sent2labels(train_sents[0]))

['B-geo', 'O', 'O', 'B-per', 'I-per', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [ ]:
print(sent2tokens(train_sents[0]))

['Iraq', "'s", 'main', 'Sunni', 'Arab', 'coalition', 'has', 'challenged', 'partial', 'vote', 'results', 'showing', 'a', 'strong', 'lead', 'for', 'a', "Shi'ite", 'coalition', 'in', 'last', 'week', "'s", 'parliamentary', 'elections', '.']


In [ ]:
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

In [ ]:
X_val = [sent2features(s) for s in val_sents]
y_val = [sent2labels(s) for s in val_sents]

### **Training**

In [ ]:
import sklearn_crfsuite

In [ ]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

In [ ]:
%%time
crf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [ ]:
import pickle

In [ ]:
with open(f"{MODEL_PATH}/sklearn/crf_ner.pkl", "wb") as f:
    pickle.dump(crf, f)

In [ ]:
with open(f"{MODEL_PATH}/sklearn/crf_ner.pkl", "rb") as f:
    loaded_crf = pickle.load(f)

### **Evaluation**

In [ ]:
from sklearn_crfsuite import metrics
import eli5

In [ ]:
labels = list(crf.classes_)
labels.remove('O')
print(labels)

['B-geo', 'B-per', 'I-per', 'B-org', 'I-geo', 'B-tim', 'I-org', 'I-tim', 'B-gpe', 'I-gpe', 'B-eve', 'I-eve', 'B-nat', 'B-art', 'I-art', 'I-nat']


In [ ]:
y_pred = crf.predict(X_val)
metrics.flat_f1_score(y_val, y_pred,
                      average='weighted', labels=labels)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1465: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


0.8595355645676034

In [ ]:
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)

# poco support
print(metrics.flat_classification_report(
    y_val, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

       B-art      0.000     0.000     0.000         6
       I-art      0.000     0.000     0.000         4
       B-eve      1.000     1.000     1.000         1
       I-eve      1.000     1.000     1.000         2
       B-geo      0.858     0.919     0.887       270
       I-geo      0.698     0.822     0.755        45
       B-gpe      0.981     0.953     0.967       106
       I-gpe      1.000     1.000     1.000         1
       B-nat      0.000     0.000     0.000         1
       I-nat      0.000     0.000     0.000         0
       B-org      0.831     0.749     0.788       171
       I-org      0.855     0.830     0.842       135
       B-per      0.895     0.797     0.843       128
       I-per      0.856     0.930     0.891       128
       B-tim      0.962     0.911     0.936       168
       I-tim      0.778     0.750     0.764        56

   micro avg      0.872     0.856     0.864      1222
   macro avg      0.670   

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=['B-art', 'I-art', 'B-eve', 'I-eve', 'B-geo', 'I-geo', 'B-gpe', 'I-gpe', 'B-nat', 'I-nat', 'B-org', 'I-org', 'B-per', 'I-per', 'B-tim', 'I-tim'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
eli5.show_weights(crf, top=30)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


From \ To,O,B-art,I-art,B-eve,I-eve,B-geo,I-geo,B-gpe,I-gpe,B-nat,I-nat,B-org,I-org,B-per,I-per,B-tim,I-tim
O,4.4,1.552,-3.558,1.766,-3.136,1.999,-5.932,1.256,-3.077,0.797,-2.385,1.849,-5.678,2.42,-4.468,2.707,-5.889
B-art,-0.327,-0.052,5.984,-0.141,-0.566,-1.318,-1.111,-2.402,-0.409,-0.053,0.0,-1.18,-1.878,-1.651,-1.598,-0.292,-1.407
I-art,-0.638,-0.495,5.489,-0.196,-0.118,-1.13,-1.099,-1.522,0.0,0.0,0.0,-1.756,-1.583,-1.713,-1.594,-1.623,-1.434
B-eve,-1.795,-0.039,-0.765,-0.908,4.688,-2.023,-1.477,-2.355,-0.412,-0.843,-0.184,-2.587,-2.095,-2.758,-1.805,-0.594,-2.059
I-eve,-0.09,0.0,-0.036,-2.696,5.318,-1.089,-0.511,-0.623,0.0,0.0,0.0,-1.132,-0.897,-1.438,-0.864,-2.175,-1.402
B-geo,0.726,0.059,-2.297,-1.466,-2.043,-4.373,5.538,-0.667,-2.786,-0.763,-0.837,-0.852,-4.321,-1.472,-4.075,1.307,-3.434
I-geo,0.249,0.595,-1.303,-0.525,-1.265,-3.29,4.522,-1.766,-1.634,-0.137,-0.319,-1.023,-3.135,-0.921,-2.677,0.524,-2.536
B-gpe,0.676,-0.855,-2.057,-0.479,-2.681,0.037,-3.577,-4.872,3.152,-0.666,-1.19,0.828,-4.783,-1.008,-3.86,-0.609,-2.918
I-gpe,-0.11,0.0,-0.0,0.0,0.0,-0.504,-1.575,-0.702,4.53,0.0,0.0,-1.686,-1.231,-2.23,-1.485,-1.242,-0.253
B-nat,0.005,0.0,0.0,0.0,0.0,-0.309,-0.115,-1.052,0.0,-0.315,7.213,-0.446,-0.182,-0.232,-1.025,0.127,-0.113


In [ ]:
raw_sent = "Argentina is located eleven thousand kilometers from England"
test_raw_sent = [sent2features(raw_sent.split())]
print(crf.predict(test_raw_sent))

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-org']]
